In [15]:
# function.py
import requests
import re
import os
from dotenv import load_dotenv
from langchain.tools import tool
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
from typing import List, Dict, Optional, Tuple


# === .env 불러오기 ===
load_dotenv()
naver_client_id = os.getenv("NAVER_CLIENT_ID")
naver_client_secret = os.getenv("NAVER_CLIENT_SECRET")


c:\Users\Playdata2\miniconda3\envs\final-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 네이버 쇼핑 API 함수

In [16]:
def price_tool(user_query: str) -> str:
    """A tool that uses the Naver Shopping API to look up perfume prices (results are returned as formatted strings)"""
    
    url = "https://openapi.naver.com/v1/search/shop.json"
    headers = {
        "X-Naver-Client-Id": naver_client_id,
        "X-Naver-Client-Secret": naver_client_secret
    }
    params = {"query": user_query, "display": 5, "sort": "sim"}
    
    try:
        response = requests.get(url, headers=headers, params=params)
    except Exception as e:
        return f"❌ 요청 오류: {e}"
    
    if response.status_code != 200:
        return f"❌ API 오류: {response.status_code}"
    
    data = response.json()
    if not data or "items" not in data or len(data["items"]) == 0:
        return f"😔 '{user_query}'에 대한 검색 결과가 없습니다."
    
    # HTML 태그 제거 함수
    def remove_html_tags(text: str) -> str:
        return re.sub(r"<[^>]+>", "", text)
    
    # 상위 3개만 정리
    products = data["items"][:3]
    output = f"🔍 '{user_query}' 검색 결과:\n\n"
    for i, item in enumerate(products, 1):
        title = remove_html_tags(item.get("title", ""))
        lprice = item.get("lprice", "0")
        mall = item.get("mallName", "정보 없음")
        link = item.get("link", "정보 없음")
        
        output += f"📦 {i}. {title}\n"
        if lprice != "0":
            output += f"   💰 가격: {int(lprice):,}원\n"
        output += f"   🏪 판매처: {mall}\n"
        output += f"   🔗 링크: {link}\n\n"
    
    return output


In [17]:
price_tool("샤넬 NO5 향수 가격 ")  # 함수 테스트

"🔍 '샤넬 NO5 향수 가격 ' 검색 결과:\n\n📦 1. CHANEL No.5 오 드 퍼퓸 플로럴향, 50ml, 1개\n   💰 가격: 130,000원\n   🏪 판매처: 네이버\n   🔗 링크: https://search.shopping.naver.com/catalog/53015716331\n\n📦 2. [국내백화점/선물포장] 샤넬 넘버5 NO5 오드빠르펭 여성 향수 35ml\n   💰 가격: 141,000원\n   🏪 판매처: 라이크컴퍼니\n   🔗 링크: https://smartstore.naver.com/main/products/11549340601\n\n📦 3. 샤넬 넘버5 오드빠르펭 1.5ml\n   💰 가격: 5,500원\n   🏪 판매처: 스완코스메틱\n   🔗 링크: https://smartstore.naver.com/main/products/5992905332\n\n"

# 메타필터링 Season

In [19]:
def best_season_tool(user_query: str) -> Optional[Dict]:
    """
    쿼리에서 계절 키워드를 추출해 Pinecone 메타필터 반환
    """
    season_map = {
        "봄": "봄",
        "여름": "여름",
        "가을": "가을",
        "autumn": "가을",
        "겨울": "겨울"
    }
    
    for kr, season_val in season_map.items():
        if kr in user_query.lower():
            return {"best_season": {"$eq": season_val}}
    
    return None


In [20]:
def test_best_season_tool():
    queries = [
        "겨울용 향수 추천해줘",
        "여름에 어울리는 시원한 향수",
        "봄 향수 추천",
        "가을에 뿌리면 좋은 향수",
        "남성용 향수 추천"   # season 키워드 없음
    ]
    
    for q in queries:
        filter_result = best_season_tool(q)
        print(f"쿼리: {q}")
        print(f"리턴값: {filter_result}\n")

# 실행
test_best_season_tool()


쿼리: 겨울용 향수 추천해줘
리턴값: {'best_season': {'$eq': '겨울'}}

쿼리: 여름에 어울리는 시원한 향수
리턴값: {'best_season': {'$eq': '여름'}}

쿼리: 봄 향수 추천
리턴값: {'best_season': {'$eq': '봄'}}

쿼리: 가을에 뿌리면 좋은 향수
리턴값: {'best_season': {'$eq': '가을'}}

쿼리: 남성용 향수 추천
리턴값: None



In [21]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

# 환경변수 로드
load_dotenv()

# Pinecone 초기화
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("perfume-search")

# 임베딩 모델
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# -------------------------------
# 1) best_season 필터 툴 함수
# -------------------------------
def best_season_tool(user_query: str):
    season_map = {
        "봄": "봄",
        "여름": "여름",
        "가을": "가을",
        "autumn": "가을",
        "겨울": "겨울"
    }
    for kr, season_val in season_map.items():
        if kr in user_query.lower():
            return {"best_season": {"$eq": season_val}}
    return None

# -------------------------------
# 2) 검색 함수
# -------------------------------
def search_perfumes(user_query: str, top_k: int = 5):
    # ① 필터 생성
    season_filter = best_season_tool(user_query)
    
    # ② 쿼리 임베딩
    query_emb = model.encode(user_query).tolist()
    
    # ③ Pinecone 검색
    results = index.query(
        vector=query_emb,
        top_k=top_k,
        include_metadata=True,
        filter=season_filter if season_filter else None
    )
    
    # ④ 결과 정리
    perfumes = []
    for m in results["matches"]:
        perfumes.append({
            "score": m["score"],
            "brand": m["metadata"].get("brand", ""),
            "name": m["metadata"].get("name", ""),
            "eng_name": m["metadata"].get("eng_name", ""),
            "best_season": m["metadata"].get("best_season", ""),
            "best_time": m["metadata"].get("best_time", ""),
            "gender": m["metadata"].get("gender", ""),
            "concentration": m["metadata"].get("concentration", ""),
            "price": m["metadata"].get("price_krw", 0),
            "detail_url": m["metadata"].get("detail_url", "")
        })
    return perfumes

# -------------------------------
# 3) 테스트 실행
# -------------------------------
if __name__ == "__main__":
    queries = [
        "겨울용 향수 추천해줘",
        "여름에 어울리는 향수 보여줘",
        "남성용 향수 추천해줘"  # season 없음 → 필터 없이 검색
    ]
    
    for q in queries:
        print(f"\n🔍 Query: {q}")
        results = search_perfumes(q, top_k=3)
        for r in results:
            print(f"- {r['brand']} {r['name']} ({r['eng_name']}) "
                  f"[시즌:{r['best_season']} | 시간:{r['best_time']} | 성별:{r['gender']}] "
                  f"💰 {r['price']}원")
            print(f"  🔗 {r['detail_url']}")



🔍 Query: 겨울용 향수 추천해줘
- 세르주 루텐 페리유스몬 보터 오 드 퍼퓸 (P?rilleusement V?tre) [시즌:겨울 | 시간:야간용 | 성별:Unisex] 💰 365000.0원
  🔗 https://www.bysuco.com/product/show/97158
- 세르주 루텐 로르프린느 오 드 퍼퓸 (L'orpheline) [시즌:겨울 | 시간:야간용 | 성별:Unisex] 💰 115000.0원
  🔗 https://www.bysuco.com/product/show/9791
- 메종 마르지엘라 웬 더 레인 스탑스 오 드 뚜왈렛 (When the Rain Stops) [시즌:겨울 | 시간:주간용 | 성별:Unisex] 💰 151420.0원
  🔗 https://www.bysuco.com/product/show/10460

🔍 Query: 여름에 어울리는 향수 보여줘
- 시슬리 코스메틱 오 드 깡빠뉴 오 드 뚜왈렛 (Eau de Campagne) [시즌:여름 | 시간:주간용 | 성별:Unisex] 💰 146700.0원
  🔗 https://www.bysuco.com/product/show/24560
- 에르메스 롬브르 드 메르베이 오 드 퍼퓸 (Lombre de Mervais) [시즌:여름 | 시간:주간용 | 성별:Unisex] 💰 175020.0원
  🔗 https://www.bysuco.com/product/show/97117
- 에르메스 롬브르 드 메르베이 오 드 퍼퓸 (Lombre de Mervais) [시즌:여름 | 시간:주간용 | 성별:Unisex] 💰 125000.0원
  🔗 https://www.bysuco.com/product/show/97117

🔍 Query: 남성용 향수 추천해줘
- 조르지오 아르마니 오 뿌르 옴므 오 드 뚜왈렛 (Eau Pour Homme) [시즌:가을 | 시간:야간용 | 성별:Male] 💰 151420.0원
  🔗 https://www.bysuco.com/product/show/693975
- 조르지오 